In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest, confint_proportions_2indep
import json

In [3]:
# Load data
df = pd.read_csv('../data/google_merch_users.csv')

desktop = df[df['device'] == 'desktop']
mobile = df[df['device'] == 'mobile']

In [ ]:
# 1. DESCRIPTIVE STATISTICS

desktop_conv = desktop['converted'].sum()
desktop_n = len(desktop)
desktop_cr = desktop_conv / desktop_n

mobile_conv = mobile['converted'].sum()
mobile_n = len(mobile)
mobile_cr = mobile_conv / mobile_n

print(f"\n Sample Summary:")
print(f"  Desktop: {desktop_n:,} users, {desktop_conv} conversions ({desktop_cr:.2%})")
print(f"  Mobile:  {mobile_n:,} users, {mobile_conv} conversions ({mobile_cr:.2%})")



📊 Sample Summary:
  Desktop: 158,917 users, 2541 conversions (1.60%)
  Mobile:  109,195 users, 1851 conversions (1.70%)


In [ ]:
# 2. STATISTICAL TEST

print(f"\n Statistical Test (Two-Sample Z-Test):")

# Check assumptions
min_success = min(desktop_conv, mobile_conv)
min_failure = min(desktop_n - desktop_conv, mobile_n - mobile_conv)

if min_success >= 5 and min_failure >= 5:
    print(f"   Assumptions met (np ≥ 5 and n(1-p) ≥ 5)")
else:
    print(f"    Small sample - consider exact test")

# Run Z-test
successes = np.array([desktop_conv, mobile_conv])
nobs = np.array([desktop_n, mobile_n])
z_stat, p_value = proportions_ztest(successes, nobs, alternative='two-sided')

print(f"\n  Z-statistic: {z_stat:.3f}")
print(f"  P-value: {p_value:.4f}")

# Decision
alpha = 0.05
if p_value < alpha:
    print(f"   SIGNIFICANT (p < {alpha})")
    significance = "Yes"
else:
    print(f"   NOT SIGNIFICANT (p ≥ {alpha})")
    significance = "No"



📈 Statistical Test (Two-Sample Z-Test):
  ✓ Assumptions met (np ≥ 5 and n(1-p) ≥ 5)

  Z-statistic: -1.928
  P-value: 0.0539
  ✗ NOT SIGNIFICANT (p ≥ 0.05)


In [6]:
# 3. EFFECT SIZE

print(f"\n Effect Size:")

abs_diff = desktop_cr - mobile_cr
rel_diff = abs_diff / mobile_cr

print(f"  Absolute: {abs_diff:.4f} ({abs_diff*100:.2f} percentage points)")
print(f"  Relative: {rel_diff:.2%} lift")

# Practical significance
if abs(rel_diff) < 0.05:
    practical_sig = "Small"
elif abs(rel_diff) < 0.20:
    practical_sig = "Medium"
else:
    practical_sig = "Large"

print(f"  Magnitude: {practical_sig}")


 Effect Size:
  Absolute: -0.0010 (-0.10 percentage points)
  Relative: -5.67% lift
  Magnitude: Medium


In [7]:
# 4. CONFIDENCE INTERVAL

ci_low, ci_high = confint_proportions_2indep(
    desktop_conv, desktop_n,
    mobile_conv, mobile_n,
    method='wald'
)

print(f"\n📏 95% Confidence Interval:")
print(f"  [{ci_low*100:.2f}pp, {ci_high*100:.2f}pp]")
print(f"  We're 95% confident desktop CR exceeds mobile by {ci_low*100:.2f}pp to {ci_high*100:.2f}pp")


📏 95% Confidence Interval:
  [-0.19pp, 0.00pp]
  We're 95% confident desktop CR exceeds mobile by -0.19pp to 0.00pp


In [8]:
# 5. BUSINESS IMPACT (THE MOST IMPORTANT PART)

print(f"\n Business Impact:")

# Calculate potential gains if mobile matched desktop
if desktop_cr > mobile_cr:
    additional_conversions = mobile_n * (desktop_cr - mobile_cr)
    
    # Revenue estimates
    avg_order_value = 50  # Typical e-commerce AOV
    monthly_revenue_gain = additional_conversions * avg_order_value
    annual_revenue_gain = monthly_revenue_gain * 12
    
    print(f"  If mobile users converted like desktop:")
    print(f"    + {additional_conversions:.0f} conversions/month")
    print(f"    + ${monthly_revenue_gain:,.0f}/month")
    print(f"    + ${annual_revenue_gain:,.0f}/year")
    
    # ROI calculation
    investment_threshold = annual_revenue_gain * 0.2  # 20% of gain
    
    print(f"\n   Justifiable investment:")
    print(f"    Up to ${investment_threshold:,.0f} for mobile optimization")
    print(f"    (20% of annual revenue gain)")
else:
    annual_revenue_gain = 0
    investment_threshold = 0


 Business Impact:


In [ ]:
# 6. CLEAR RECOMMENDATION

if p_value < alpha and abs(rel_diff) > 0.10:
    recommendation = "STRONG: Invest in optimization"
    action = "Prioritize mobile UX improvements (checkout flow, page speed)"
elif p_value < alpha:
    recommendation = "MODERATE: Consider optimization"
    action = "Test specific mobile improvements (A/B test mobile-only features)"
else:
    recommendation = "WEAK: Monitor, don't invest yet"
    action = "Gather more data or focus on other opportunities"

print(f"\n  Decision: {recommendation}")
print(f"  Action: {action}")


🎯 RECOMMENDATION

  Decision: WEAK: Monitor, don't invest yet
  Action: Gather more data or focus on other opportunities


In [10]:
# 7. SAVE RESULTS

results = {
    # Sample
    'desktop_n': int(desktop_n),
    'mobile_n': int(mobile_n),
    'desktop_conversions': int(desktop_conv),
    'mobile_conversions': int(mobile_conv),
    
    # Rates
    'desktop_cr': float(desktop_cr),
    'mobile_cr': float(mobile_cr),
    
    # Test
    'z_statistic': float(z_stat),
    'p_value': float(p_value),
    'significant': bool(p_value < alpha),
    'alpha': alpha,
    
    # Effect
    'absolute_difference': float(abs_diff),
    'relative_lift': float(rel_diff),
    'practical_significance': practical_sig,
    
    # CI
    'ci_lower': float(ci_low),
    'ci_upper': float(ci_high),
    
    # Business
    'annual_revenue_impact': float(annual_revenue_gain),
    'investment_threshold': float(investment_threshold),
    
    # Decision
    'recommendation': recommendation,
    'action': action
}

with open('../results/test_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n Results saved")


 Results saved
